In [146]:
import cv2
import torch
import numpy as np

In [147]:
def ratio_overlap_parking_area(p, d):
    '''
    Calculating ratio of area of parking spot, 
    which is overlapped by detected object
    
    p: (xp1, yp1, xp2, yp2) - tuple of left upper coordinate and
    bottom right coordinate of parking spot
    d: (xd1, yd1, xd2, yd2) - tuple of left upper coordinate and
    bottom right coordinate of detected object
    '''
    xp1, yp1, xp2, yp2 = p
    xd1, yd1, xd2, yd2 = d
    dx = max(0, min(xp2, xd2) - max(xp1, xd1))
    dy = max(0, min(yp2, yd2) - max(yp1, yd1))
    opa = dx * dy
    area = (xp2 - xp1) * (yp2 - yp1)
    return opa / area

assert abs(ratio_overlap_parking_area((0, 2, 5, 4), (2, 1, 4, 5)) - 0.4) < 1e-6

In [148]:
def frame_processing(frame, grid, model, t=0.33):
    '''
    Estimating load of parking on the frame
    
    frame: Numpy array; Picture of parking area
    grid: Numpy array; Coordinates of all parking spots
    model: torch.nn.Module; Model for object detection task
    t: float; threshold 
    '''
    results = model(frame)
    d_coords = results.pred[0].numpy()[:, :4].astype(np.int32)
    is_available = [True] * len(grid)
    for i, p in enumerate(grid):
        for d in d_coords:
            r = ratio_overlap_parking_area(p, d)
            if r > t:
                is_available[i] = False
                continue
    return is_available, d_coords

In [149]:
def video_processing(source, grid, model, t=0.33):
    '''
    Estimating load of parking on the video stream
    
    source: cv2.VideoCapture object; videostream from parking area
    grid: Numpy array; Coordinates of all parking spots
    model: torch.nn.Module; Model for object detection task
    t: float; threshold
    '''
    vrf = cv2.VideoWriter_fourcc(*'MP4V')
    vr = cv2.VideoWriter('result-full.mp4', vrf, 30.0, (720, 1280))
    while source.isOpened():
        # Capture frame-by-frame
        ret, frame = source.read()
        if ret == True:
            available_spots, d_coords = frame_processing(frame, grid, model, t)
            img = frame.copy()
            for p, a in zip(grid, available_spots):
                x1, y1, x2, y2 = p
                color = (0, 255, 0) if a else (0, 0, 255)
                img = cv2.rectangle(img, (x1, y1), (x2, y2), color=color, thickness=2)
            vr.write(img)
        else:
            break

In [150]:
%%time
# Model
model = torch.hub.load('ultralytics/yolov5', 'yolov5n', device='cpu')  # or yolov5n - yolov5x6, custom
# Classes for vehicles:
# car: 2
# motorcycle: 3
# bus: 5
# truck: 7
# model.classes = [2, 3, 5, 7]

grid = []
with open('grid.csv', 'r') as f:
    for line in f:
        grid.append(list(map(int, line.strip().split('\t'))))
grid = np.array(grid)

source = cv2.VideoCapture('trehgorka.mp4')

video_processing(source, grid, model)

Using cache found in /home/aleksandr/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2022-9-22 Python-3.8.3 torch-1.12.1+cu113 CPU

Fusing layers... 
YOLOv5n summary: 213 layers, 1867405 parameters, 0 gradients
Adding AutoShape... 


CPU times: user 4min 46s, sys: 324 ms, total: 4min 47s
Wall time: 47.3 s


In [ ]:
# YOLO size   time
# n           47.3s
# s           1min 43s
# m           3min 11s